In [1]:
import pandas as pd
import numpy as np
from pyecharts import Bar, Line, Pie

C:\Users\user\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data = pd.read_csv('netflix_titles.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [4]:
data.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [55]:
for i in range(len(data)):
    try:
        data.loc[i, 'country'] = data.loc[i, 'country'].replace("\"", "")
    except:
        pass

In [56]:
countries = []
for i in range(len(data)):
    if type(data.loc[i, 'country']) == str:
        countries_content = data.loc[i, 'country'].split(', ')
        for country in countries_content:
            countries.append(country)
    else:
        pass

countries = list(set(countries))

In [68]:
#countries.remove('')
countires_re = [ country.replace(',', '') for country in countries ]

In [70]:
countires_re = list(set(countires_re))

In [71]:
countries_cnt = []
for country in countires_re:
    cnt = 0
    for i in range(len(data)):
        if type(data.loc[i, 'country']) == str and data.loc[i, 'country'].__contains__(country):
            cnt +=1
    countries_cnt.append(cnt) 

In [72]:
countries_movie_number = pd.DataFrame({'country':countires_re, 'count':countries_cnt})
countries_movie_number.to_csv('국가별_콘텐츠_개수.csv')

In [81]:
countries_movie_number.sort_values(by='count', ascending=False)[:10]

,country,count
70,United States,3690
94,India,1046
61,United Kingdom,806
66,Canada,445
16,France,393
64,Japan,318
42,Spain,232
98,South Korea,231
25,Germany,231
106,Mexico,169


In [82]:
data['release_year']

0       2020
1       2021
2       2021
3       2021
4       2021
        ... 
8802    2007
8803    2018
8804    2009
8805    2006
8806    2015
Name: release_year, Length: 8807, dtype: int64

In [87]:
for i in range(len(data)):
    if type(data.loc[i, 'date_added']) == str:
        data.loc[i, 'date_added_year'] = data.loc[i, 'date_added'][-4:]

In [88]:
for i in range(len(data)):
    if data.loc[i, 'release_year'] <= 1980:
        data.loc[i, 'before_1980'] = 1
    else:
        data.loc[i, 'before_1980'] = 0

In [96]:
pd.pivot_table(data,  index=['date_added_year'], columns=['before_1980'],  aggfunc='count')['title']

before_1980,0.0,1.0
date_added_year,,
2008,2.0,NaN
2009,2.0,NaN
2010,1.0,NaN
2011,13.0,NaN
2012,3.0,NaN
2013,11.0,NaN
2014,24.0,NaN
2015,82.0,NaN
2016,427.0,2.0


In [ ]:
# 카테고리 분석

In [5]:
data.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [7]:
categories = []

for i in range(len(data)):
    cats = data.loc[i, 'listed_in'].split(', ')
    for cat in cats:
        categories.append(cat)

categories = set(categories)

In [9]:
show_ids = []
categories_list = []

for i in range(len(data)):
    cats = data.loc[i, 'listed_in'].split(', ')
    for cat in cats:
        show_ids.append(data.loc[i, 'show_id'])
        categories_list.append(cat)

In [11]:
show_category = pd.DataFrame({'show_id':show_ids, 'category':categories_list})

In [22]:
bar = Bar("")
bar.add("category", show_category.category.value_counts()[:10].keys(), show_category.category.value_counts()[:10].values, is_stack=True)
bar